<a href="https://colab.research.google.com/github/chumpblocckami/TextGenerator/blob/master/TextGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
class TextGenerator():
  nltk.download("punkt")
  def getTextFromWikipedia(self,link):
    '''
      get the text from a url page (better use Wikipedia)
    '''
    req = requests.get(link).text
    soup = BeautifulSoup(req)
    text = []
    for i in soup.find_all("p"):
      text.append(i.text)
    return text

  def Preprocessing(self,text):
    '''
      apply simple preprocessing
    '''
    text_preproc = " ".join(nltk.word_tokenize(" ".join(text))).lower()
    text_preproc = [x for x in text_preproc.split(" ") if x not in string.punctuation]
    return text_preproc

  def Simple(self,text,lenght_sentence=10):
    '''
      generate text randomly
    '''
    counter = dict(Counter(text))
    sentence = "/begin"
    for i in range(0,lenght_sentence):
      choice = random.randint(0,len(counter.keys()))
      choice_word = list(counter.keys())[choice]
      sentence = sentence + " " +choice_word
    sentence = sentence + "/end"
    return sentence

  def MostFrequent(self,text,lenght_sentence=10):
    '''
      generate text using weights based on most common words
    '''
    counter = dict(Counter(text))
    sentence = "/begin"
    sums = sum(counter.values())
    weights = [round(x/sums,5) for x in counter.values()]
    for i in range(0,lenght_sentence):
      choice_word = random.choices(population=list(counter.keys()),weights=weights, k=1)
      sentence = sentence + " " + "".join(choice_word)
    sentence = sentence+"/end"
    return sentence

    #HMM random words based on co-occurrence matrix
  def getCoOccurence(self,text,step=2):
    '''
      Create a co-occurrence dicts where keys are the target words and values are the words that follows.
      Step is the size window, so if step is 2, it detect only the following words, if step is 3, it detect the two following words, ecc...
    '''
    co_occurrence_matrix = {}
    vocab = list(set(text))
    for word in vocab:
      cache = []
      pos = 0
      for i in text:
        if (i == word):
          for count in range(1,step):
            try:
              cache.append(text[pos+count])
            except:
              pass
              break
        else:
          pass
        pos = pos+1
      if(word in co_occurrence_matrix.keys()):
        tot =  co_occurrence_matrix.extend(cache)
        co_occurrence_matrix[word].update(tot)
      else:
        co_occurrence_matrix[word] = cache
    return co_occurrence_matrix

  def HiddenMarkovModel(self,text,lenght_sentence=10,co_step=2):
    '''
      get random text based on hidden markov model, using generated state based on co-occurrence matrix
    '''
    sentence = "/begin"
    co_matrix = self.getCoOccurence(text_preproc,co_step=2)
    word = random.choice(list(co_matrix.keys()))
    for i in range(0,lenght_sentence):
      sentence = sentence + " " + "".join(word)
      try:
        word = random.choice(co_matrix[word])
      except:
        print("Word '"+word+"' does not have any co-occurrence!\nSkipping...")
        pass
    sentence = sentence+" /end"
    return sentence